In [ ]:
import re
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
from tqdm import tqdm

# -----------------------------------------------------------------------------
# 1) Precompile Regexes & Setup Constants
# -----------------------------------------------------------------------------

# match pure-number → Excel serial
RE_SERIAL = re.compile(r'^\d+(\.\d+)?$')
# match text dates like "12-13-2010 9:02" or "12-01-2010 08:26:00"
RE_TEXT   = re.compile(r'^\d{1,2}-\d{1,2}-\d{4}\s+\d{1,2}:\d{2}(:\d{2})?$')

# Excel's zero; pandas uses the same convention
EXCEL_ORIGIN = datetime(1899, 12, 30)

def serials_to_datetime(arr: np.ndarray) -> pd.DatetimeIndex:
    """
    Convert an array of floats (Excel serial dates) into a DatetimeIndex.
    """
    # pandas-friendly vectorized conversion
    return EXCEL_ORIGIN + pd.to_timedelta(arr.astype(float), unit='D')

# -----------------------------------------------------------------------------
# 2) Load CSV (force InvoiceDate → str so serials don’t auto-convert to floats)
# -----------------------------------------------------------------------------

df = pd.read_csv("/content/invoices_cleaned.csv", dtype={"InvoiceDate": str})

# strip whitespace once
df["InvoiceDate"] = df["InvoiceDate"].str.strip()

# -----------------------------------------------------------------------------
# 3) Identify masks
# -----------------------------------------------------------------------------

is_serial = df["InvoiceDate"].str.match(RE_SERIAL)
is_text   = df["InvoiceDate"].str.match(RE_TEXT)

# -----------------------------------------------------------------------------
# 4) Vectorized conversions
# -----------------------------------------------------------------------------

# A) Excel-serials → datetime
if is_serial.any():
    serial_vals = df.loc[is_serial, "InvoiceDate"].astype(float).to_numpy()
    df.loc[is_serial, "InvoiceDate"] = serials_to_datetime(serial_vals)

# B) Well-formed text dates → datetime via pandas.to_datetime with format
#    (this is still vectorized under the hood)
if is_text.any():
    df.loc[is_text, "InvoiceDate"] = pd.to_datetime(
        df.loc[is_text, "InvoiceDate"],
        format="%m-%d-%Y %H:%M:%S",
        errors="coerce"
    )

# -----------------------------------------------------------------------------
# 5) Fallback: any remaining that aren’t datetime yet
# -----------------------------------------------------------------------------

# find rows still not converted
needs_fixing = ~pd.to_datetime(df["InvoiceDate"], errors="coerce").notna()

if needs_fixing.any():
    tqdm.pandas(desc="Parsing fallback dates")

    def parse_fallback(s: str) -> pd.Timestamp:
        """Try regex+strptime, else pandas to_datetime."""
        if RE_TEXT.match(s):
            # normalize hh:mm or hh:mm:ss
            m = RE_TEXT.match(s)
            parts = m.group(0).split()
            dt = datetime.strptime(parts[0] + " " + parts[1], "%m-%d-%Y %H:%M:%S")
            return pd.Timestamp(dt)
        # last resort
        return pd.to_datetime(s, errors="coerce")

    df.loc[needs_fixing, "InvoiceDate"] = (
        df.loc[needs_fixing, "InvoiceDate"]
          .progress_apply(parse_fallback)
    )

# -----------------------------------------------------------------------------
# 6) Final cleanup & save
# -----------------------------------------------------------------------------

# Report any that _still_ failed
n_failed = df["InvoiceDate"].isna().sum()
if n_failed:
    print(f"⚠️ {n_failed} rows could not be parsed (set to NaT)")


# Ensure the whole column really is datetime‐typed before using .dt
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# (Optional) format uniformly as string for CSV / DB import
df["InvoiceDate"] = df["InvoiceDate"].dt.strftime("%Y-%m-%d %H:%M:%S")

df.to_csv("invoices_fixed.csv", index=False)
print("✅ Done! → invoices_fixed.csv")


Parsing fallback dates: 100%|██████████| 229253/229253 [01:53<00:00, 2022.31it/s]


✅ Done! → invoices_fixed.csv
